Erst eine virtual enviroment setupen für python

In [6]:
! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python gpt4all langchain-nomic bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [ ]:
! ollama pull llama3

In [2]:
### LLM

local_llm = "llama3"

In [7]:
### Index
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

urls = [
    "https://wiki.prooph-board.com/",
    "https://wiki.prooph-board.com/event_storming/what-is-event-storming.html",
    "https://wiki.prooph-board.com/event_modeling/why-event-modeling.html",
    "https://wiki.prooph-board.com/cody/introduction.html",
    "https://wiki.prooph-board.com/cody/Cody-Server.html",
    "https://wiki.prooph-board.com/cody/nodejs-cody-tutorial.html",
    "https://wiki.prooph-board.com/cody/php-cody-tutorial.html",
    "https://wiki.prooph-board.com/cody_play/tutorial.html",
    "https://wiki.prooph-board.com/cody_engine/introduction.html",
    "https://wiki.prooph-board.com/cody_engine/information-schema.html",
    "https://wiki.prooph-board.com/board_workspace/Board-API.html",
    "https://wiki.prooph-board.com/board_workspace/Cards.html",
    "https://wiki.prooph-board.com/board_workspace/Cody-Suggestions.html",
    "https://wiki.prooph-board.com/board_workspace/Event-Modeling-Mode.html",
    "https://wiki.prooph-board.com/board_workspace/Frames.html",
    "https://wiki.prooph-board.com/board_workspace/Keymap.html",
    "https://wiki.prooph-board.com/board_workspace/Lite-Mode.html",
    "https://wiki.prooph-board.com/board_workspace/Metadata.html",
    "https://wiki.prooph-board.com/board_workspace/Tree-View.html",
    "https://wiki.prooph-board.com/access_management/Access-Rights.html",
    "https://wiki.prooph-board.com/access_management/Managing-an-Organization.html",


]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

print(doc_splits)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)
print(vectorstore)
retriever = vectorstore.as_retriever()

[Document(page_content='prooph board Wiki · prooph board Wiki\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nby prooph software\n\n\n\n\n\n\nGetting Started\n\n\n... as Agile Coach\n\n\n... as Architect\n\n\n... as C-Level\n\n\n... as Data Engineer\n\n\n... as DevOp\n\n\n... as Developer\n\n\n... as Domain Expert\n\n\n... as Product Owner\n\n\n... as Project Manager\n\n\n... as QA\n\n\n... as SRE\n\n\n... as UX/UI\n\n\nEvent Storming Basics\n\n\nWhat is Event Storming\n\n\nBasic Concepts\n\n\nPrepare Big Picture Session\n\n\nEvent Modeling\n\n\nWhy Event Modeling\n\n\nPhilosophy\n\n\nHow To\n\n\n... on prooph board\n\n\nTips and Tricks\n\n\nCody - The Coding Bot\n\n\nIntroduction\n\n\nServer Implementation\n\n\nNodeJS Cody Tutorial\n\n\nPHP Cody Tutorial\n\n\nCody Play\n\n\nA Playground for your Design\n\n\nTutorial\n\n\nDocs\n\n\nPlayshots\n\n\nCody Engine (beta)\n\n\nCody Engine Intro\n\n\nInformation Schema\n\n\nBoard Workspace\n\n\nBoard API\n\n\nWorking With Cards', metadata={'source': 

In [ ]:
### Generate

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "Can u tell me about frames?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)